In [ ]:
import sys, os
if 'google.colab' in sys.modules and not os.path.exists('.setup_complete'):
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/setup_colab.sh -O- | bash
    
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week09_policy_II/runners.py
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week09_policy_II/mujoco_wrappers.py
    
    !touch .setup_complete

# This code creates a virtual display to draw game images on.
# It will have no effect if your machine has a monitor.
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    os.environ['DISPLAY'] = ':1'

# Implementing Proximal Policy Optimization 


In this notebook you will be implementing Proximal Policy Optimization algorithm, 
scaled up version of which was used to train [OpenAI Five](https://openai.com/blog/openai-five/) 
to [win](https://openai.com/blog/how-to-train-your-openai-five/) against the
world champions in Dota 2.
You will be solving a continuous control environment on which it may be easier and faster 
to train an agent, however note that PPO here may not be the best algorithm as, for example,
Deep Deterministic Policy Gradient and Soft Actor Critic may be more suited 
for continuous control environments. To run the environment you will need to install 
[pybullet-gym](https://github.com/benelot/pybullet-gym) which unlike MuJoCo 
does not require you to have a license.

To install the library:

In [ ]:
!git clone https://github.com/benelot/pybullet-gym lib/pybullet-gym
!pip install -e lib/pybullet-gym
!pip install wandb

The overall structure of the code is similar to the one in the A2C optional homework, but don't worry if you haven't done it, it should be relatively easy to figure it out. 
First, we will create an instance of the environment. 
We will normalize the observations and rewards, but before that you will need a wrapper that will 
write summaries, mainly, the total reward during an episode. You can either use one for `TensorFlow` 
implemented in `atari_wrappers.py` file from the optional A2C homework, or implement your own. 

In [ ]:
CWD = './'

import gym 
import pybulletgym

env = gym.make("HalfCheetahMuJoCoEnv-v0")
print("observation space: ", env.observation_space,
      "\nobservations:", env.reset())
print("action space: ", env.action_space, 
      "\naction_sample: ", env.action_space.sample())

In [ ]:
import wandb
from collections import defaultdict, deque
import numpy as np


class SummariesBase(gym.Wrapper):
    """ Env summaries writer base."""

    def __init__(self, env, prefix=None, running_mean_size=100):
        super().__init__(env)
        self.episode_counter = 0
        self.prefix = prefix

        nenvs = getattr(self.env.unwrapped, "nenvs", 1)
        self.rewards = np.zeros(nenvs)
        self.had_ended_episodes = np.zeros(nenvs, dtype=np.bool)
        self.episode_lengths = np.zeros(nenvs)
        self.reward_queues = [deque([], maxlen=running_mean_size)
                              for _ in range(nenvs)]

    def should_write_summaries(self):
        """ Returns true if it's time to write summaries. """
        return np.all(self.had_ended_episodes)

    def add_summaries(self):
        """ Writes summaries. """
        self.add_summary_scalar(
            f"{self.prefix}/total_reward",
            np.mean([q[-1] for q in self.reward_queues]))
        self.add_summary_scalar(
            f"{self.prefix}/reward_mean_{self.reward_queues[0].maxlen}",
            np.mean([np.mean(q) for q in self.reward_queues]))
        self.add_summary_scalar(
            f"{self.prefix}/episode_length",
            np.mean(self.episode_lengths))
        if self.had_ended_episodes.size > 1:
            self.add_summary_scalar(
                f"{self.prefix}/min_reward",
                min(q[-1] for q in self.reward_queues))
            self.add_summary_scalar(
                f"{self.prefix}/max_reward",
                max(q[-1] for q in self.reward_queues))
        self.episode_lengths.fill(0)
        self.had_ended_episodes.fill(False)

    def step(self, action):
        obs, rew, done, info = self.env.step(action)
        self.rewards += rew
        self.episode_lengths[~self.had_ended_episodes] += 1

        info_collection = [info] if isinstance(info, dict) else info
        done_collection = [done] if isinstance(done, bool) else done
        done_indices = [i for i, info in enumerate(info_collection)
                        if info.get("real_done", done_collection[i])]
        for i in done_indices:
            if not self.had_ended_episodes[i]:
                self.had_ended_episodes[i] = True
            self.reward_queues[i].append(self.rewards[i])
            self.rewards[i] = 0

        if self.should_write_summaries():
            self.add_summaries()
        return obs, rew, done, info

    def reset(self, **kwargs):
        self.rewards.fill(0)
        self.episode_lengths.fill(0)
        self.had_ended_episodes.fill(False)
        return self.env.reset(**kwargs)

    
class TorchSummaries(SummariesBase):
    """ Wrapper to write summaries. """
    def __init__(self, env, prefix=None, running_mean_size=100):
        super().__init__(env, prefix, running_mean_size)
        self.__step = 0
        
    def set_step(self, step):
        self.__step = step

    def add_summary_scalar(self, name, value, step=None):
        if step is None:
            step = self.__step
        wandb.log({name: value}, step=step)

The normalization wrapper will subtract running mean from observations and rewards and divide 
the resulting quantities by the  running variances.

In [ ]:
from mujoco_wrappers import Normalize

ENV_NAME = "HalfCheetahMuJoCoEnv-v0"

def create_mujoco_env():
    env = gym.make(ENV_NAME)
    return env

NENVS = 1
env = Normalize(TorchSummaries(create_mujoco_env(), prefix=ENV_NAME))

STATE_DIM = env.observation_space.shape[0]
N_ACTIONS = env.action_space.shape[0]

In [ ]:
class AsArray:
    """ 
    Converts lists of interactions to ndarray.
    """
    def __call__(self, trajectory):
        # Modify trajectory inplace. 
        for k, v in filter(lambda kv: kv[0] != "state",
                           trajectory.items()):
            trajectory[k] = np.asarray(v)
            
class AsTensor:
    """ 
    Converts lists of interactions to DEVICE torch.Tensor.
    """
    def __call__(self, trajectory):
        # Modify trajectory inplace. 
        for k, v in filter(lambda kv: kv[0] != "state",
                           trajectory.items()):
            trajectory[k] = torch.Tensor(v).to(DEVICE)

""" RL env runner """
from collections import defaultdict

import numpy as np


class EnvRunner:
    """ Reinforcement learning runner in an environment with given policy """

    def __init__(self, env, policy, nsteps, transforms=None, step_var=None):
        self.env = env
        self.policy = policy
        self.nsteps = nsteps
        self.transforms = transforms or []
        self.step_var = step_var if step_var is not None else 0
        self.state = {"latest_observation": self.env.reset()}

    @property
    def nenvs(self):
        """ Returns number of batched envs or `None` if env is not batched """
        return getattr(self.env.unwrapped, "nenvs", None)

    def reset(self):
        """ Resets env and runner states. """
        self.state["latest_observation"] = self.env.reset()
        self.policy.reset()

    def get_next(self):
        """ Runs the agent in the environment.  """
        trajectory = defaultdict(list, {"actions": []})
        observations = []
        rewards = []
        resets = []
        self.state["env_steps"] = self.nsteps

        for i in range(self.nsteps):
            observations.append(self.state["latest_observation"])
            act = self.policy.act(self.state["latest_observation"])
            if "actions" not in act:
                raise ValueError("result of policy.act must contain 'actions' "
                                 f"but has keys {list(act.keys())}")
            for key, val in act.items():
                trajectory[key].append(val)

            obs, rew, done, _ = self.env.step(trajectory["actions"][-1])
            self.state["latest_observation"] = obs
            rewards.append(rew)
            resets.append(done)
            self.step_var += self.nenvs or 1

            # Only reset if the env is not batched. Batched envs should
            # auto-reset.
            if not self.nenvs and np.all(done):
                self.state["env_steps"] = i + 1
                self.state["latest_observation"] = self.env.reset()

        trajectory.update(
            observations=observations,
            rewards=rewards,
            resets=resets)
        trajectory["state"] = self.state

        for transform in self.transforms:
            transform(trajectory)
        return trajectory

We will use `EnvRunner` to perform interactions with an environment with a policy for a fixed number of timesteps. Calling `.get_next()` on a runner will return a trajectory &mdash; dictionary 
containing keys

* `"observations"`
* `"rewards"` 
* `"resets"`
* `"actions"`
* all other keys that you defined in `Policy`,

under each of these keys there is a `np.ndarray` of specified length $T$ &mdash; the size of partial trajectory. 

Additionally, before returning a trajectory this runner can apply a list of transformations. 
Each transformation is simply a callable that should modify passed trajectory in-place.

In [ ]:
import numpy as np

class DummyPolicy:
    def act(self, inputs, training=False):
        assert not training
        N = inputs.shape[0]
        return {"actions": np.random.randn(6,), "values": np.nan * np.ones(N)}

runner = EnvRunner(env, DummyPolicy(), 3,
                   transforms=[AsArray()])
trajectory = runner.get_next()

{k: v.shape for k, v in trajectory.items() if k != "state"}

Next, you will need to define a model for training. We suggest that you use two separate networks: one for policy
and another for value function. Each network should be a 3-layer MLP with 64 hidden units, $\mathrm{tanh}$ 
activation function, kernel matrices initialized with orthogonal initializer with parameter $\sqrt{2}$
and biases initialized with zeros. 

Our policy distribution is going to be multivariate normal with diagonal covariance. 
The network from above will predict the mean, and the covariance should be represented by a single 
(learned) vector of size 6 (corresponding to the dimensionality of the action space from above). 
You should initialize this vector to zero and take the exponent of it to always
have a non-negative quantity. 

Overall the model should return three things: predicted mean of the distribution, variance vector, 
value function. 

In [ ]:
import torch
from torch import nn

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'


class PPOAgent(nn.Module):
    def __init__(self):
        super().__init__()
        self.policy_nn = nn.Sequential(nn.Linear(STATE_DIM, 64),
                                       nn.ReLU(),
                                       nn.Linear(64, 64),
                                       nn.ReLU(),
                                       nn.Linear(64, N_ACTIONS),
                                       nn.Tanh())
        self.value_nn = nn.Sequential(nn.Linear(STATE_DIM, 64),
                                      nn.ReLU(),
                                      nn.Linear(64, 64),
                                      nn.ReLU(),
                                      nn.Linear(64, 1))
        self.covariance_vec = nn.Parameter(torch.zeros(N_ACTIONS, requires_grad=True))
        self.__initialize_net_weights(self.policy_nn)
        self.__initialize_net_weights(self.value_nn)

    def __initialize_net_weights(self, net):
        for p in net.parameters():
            if p.ndim < 2:
                nn.init.zeros_(p)
            else:
                nn.init.orthogonal_(p, 2 ** 0.5)
                
    def forward(self, observations):
        policy_mean = self.policy_nn(observations)
        value = self.value_nn(observations)
        return policy_mean, torch.exp(self.covariance_vec), value

This model will be wrapped by a `Policy`. The policy can work in two modes, but in either case 
it is going to return dictionary with string-type keys. The first mode is when the policy is 
used to sample actions for a trajectory which will later be used for training. In this case 
the flag `training` passed to `act` method is `False` and the method should return 
a `dict` with the following keys: 

* `"actions"`: actions to pass to the environment
* `"log_probs"`: log-probabilities of sampled actions
* `"values"`: value function $V^\pi(s)$ predictions.

We don't need to use the values under these keys for training, so all of them should be of type `np.ndarray`.

When `training` is `True`, the model is training on a given batch of observations. In this
case it should return a `dict` with the following keys

* `"distribution"`: an instance of multivariate normal distribution (`torch.distributions.MultivariateNormal` or `tf.distributions.MultivariateNormalDiag`)
* `"values"`: value function $V^\pi(s)$ prediction.

The distinction about the modes comes into play depending on where the policy is used: if it is called from `EnvRunner`, 
the `training` flag is `False`, if it is called from `PPO`, the `training` flag is `True`. These classed 
will be described below. 

In [ ]:
class Policy:
    def __init__(self, model):
        self.model = model
        
    def estimate_v(self, inputs):
        with torch.no_grad():
            return self.model.value_nn(inputs)
    
    def act(self, inputs, training=False):
        # Should return a dict.
        if isinstance(inputs, np.ndarray):
            inputs = torch.Tensor(inputs).to(DEVICE)
        if training:
            mean, cov, value = self.model(inputs)
            return {'distribution': torch.distributions.MultivariateNormal(mean, torch.diag(cov)),
                    'values': value}
        else:
            with torch.no_grad():
                mean, cov, value = self.model(inputs)
                dist = torch.distributions.MultivariateNormal(mean, torch.diag(cov))
                actions = dist.sample()
                return {'actions': actions.cpu().numpy(),
                        'log_probs': dist.log_prob(actions).cpu().numpy(),
                        'values': value.cpu().numpy()}

You will need to implement the following two transformations. 

The first is `GAE` that implements [Generalized Advantage Estimator](https://arxiv.org/abs/1506.02438).
In it you should add two keys to the trajectory: `"advantages"` and `"value_targets"`. In GAE the advantages
$A_t^{\mathrm{GAE}(\gamma,\lambda)}$ are essentially defined as the exponential 
moving average with parameter $\lambda$ of the regular advantages 
$\hat{A}^{(T)}(s_t) = \sum_{l=0}^{T-1-t} \gamma^l r_{t+l} + \gamma^{T} V^\pi(s_{T}) - V^\pi(s_t)$. 
The exact formula for the computation is the following

$$
A_{t}^{\mathrm{GAE}(\gamma,\lambda)} = \sum_{l=0}^{T-1-t} (\gamma\lambda)^l\delta_{t + l}^V, \, t \in [0, T)
$$
where $\delta_{t+l}^V = r_{t+l} + \gamma V^\pi(s_{t+l+1}) - V^\pi(s_{t+l})$. You can look at the 
derivation (formulas 11-16) in the paper. Don't forget to reset the summation on terminal
states as determined by the flags `trajectory["resets"]`. You can use `trajectory["values"]`
to get values of all observations except the most recent which is stored under 
 `trajectory["state"]["latest_observation"]`. For this observation you will need to call the policy 
 to get the value prediction.

Once you computed the advantages, you can get the targets for training the value function by adding 
back values:
$$
\hat{V}(s_{t+l}) = A_{t+l}^{\mathrm{GAE}(\gamma,\lambda)} + V(s_{t + l}),
$$
where $\hat{V}$ is a tensor of value targets that are used to train the value function. 

In [ ]:
class GAE:
    """ Generalized Advantage Estimator. """
    def __init__(self, policy, gamma=0.99, lambda_=0.95):
    self.policy = policy
    self.gamma = gamma
    self.lambda_ = lambda_

    def __call__(self, trajectory):
    <TODO: implement>

The main advantage of PPO over simpler policy based methods like A2C is that it is possible
to train on the same trajectory for multiple gradient steps. The following class wraps 
an `EnvRunner`. It should call the runner to get a trajectory, then return minibatches 
from it for a number of epochs, shuffling the data before each epoch.

In [ ]:
from math import ceil
from tqdm.notebook import tqdm

class TrajectorySampler:
    """ Samples minibatches from trajectory for a number of epochs. """
    def __init__(self, runner, num_epochs, num_minibatches, transforms=None):
        self.runner = runner
        self.num_epochs = num_epochs
        self.num_minibatches = num_minibatches
        self.transforms = transforms or []
        self.minibatch_count = 0
        self.epoch_count = 0
        self.trajectory = None
        self.trajectory_length = 0
        self.permutation = None
        self.sample_trajectory()
        
    def sample_trajectory(self):
        self.trajectory = self.runner.get_next()
        self.trajectory_length = self.trajectory['actions'].shape[0]
        self.shuffle_trajectory()
                    
    def choose_minibatch(self, idx):
        mb_size = ceil(self.trajectory_length / self.num_minibatches)
        permutation_slice = self.permutation[idx*mb_size:(idx+1)*mb_size]
        minibatch = {}
        for k, v in self.trajectory.items():
            if k != 'state':
                minibatch[k] = v[permutation_slice]
        return minibatch
    
    def shuffle_trajectory(self):
        """ Shuffles all elements in trajectory.

        Should be called at the beginning of each epoch.
        """
        self.permutation = torch.randperm(self.trajectory_length)
    
    def get_next(self):
        """ Returns next minibatch.  """
        if self.minibatch_count == self.num_minibatches:
            self.minibatch_count = 0
            self.epoch_count += 1
            self.shuffle_trajectory()
        if self.epoch_count == self.num_epochs:
            self.epoch_count = 0
            self.sample_trajectory()
        minibatch = self.choose_minibatch(self.minibatch_count)
        self.minibatch_count += 1
        for tf in self.transforms:
            tf(minibatch)
        return minibatch

A common trick to use with GAE is to normalize advantages, the following transformation does that. 

In [ ]:
class NormalizeAdvantages:
    """ Normalizes advantages to have zero mean and variance 1. """
    def __call__(self, trajectory):
    adv = trajectory["advantages"]
    adv = (adv - adv.mean()) / (adv.std() + 1e-8)
    trajectory["advantages"] = adv

Finally, we can create our PPO runner. 

In [ ]:
def make_ppo_runner(env, policy, num_runner_steps=2048,
                    gamma=0.99, lambda_=0.95, 
                    num_epochs=10, num_minibatches=32):
    """ Creates runner for PPO algorithm. """
    runner_transforms = [AsArray(),
                       GAE(policy, gamma=gamma, lambda_=lambda_)]
    runner = EnvRunner(env, policy, num_runner_steps, 
                     transforms=runner_transforms)

    sampler_transforms = [NormalizeAdvantages()]
    sampler = TrajectorySampler(runner, num_epochs=num_epochs, 
                              num_minibatches=num_minibatches,
                              transforms=sampler_transforms)
    return sampler

In the next cell you will need to implement Proximal Policy Optimization algorithm itself. The algorithm
modifies the typical policy gradient loss in the following way:

$$
J_{\pi}(s, a) = \frac{\pi_\theta(a|s)}{\pi_\theta^{\text{old}}(a|s)} \cdot A^{\mathrm{GAE}(\gamma,\lambda)}(s, a)
$$

$$
J_{\pi}^{\text{clipped}}(s, a) = \mathrm{clip}\left(
\frac{\pi_\theta(a|s)}{\pi_{\theta^{\text{old}}}(a|s)},
1 - \text{cliprange}, 1 + \text{cliprange}\right)\cdot A^{\mathrm{GAE(\gamma, \lambda)}}(s)\\
$$

$$
L_{\text{policy}} = -\frac{1}{T}\sum_{l=0}^{T-1}\min\left(J_\pi(s_{t + l}, a_{t + l}), J_{\pi}^{\text{clipped}}(s_{t + l}, a_{t + l})\right).
$$

The value loss is also modified:

$$
L_{V}^{\text{clipped}} = \frac{1}{T}\sum_{l=0}^{T-1} \max(l^{simple}(s_{t + l}), l^{clipped}(s_{t + l}))
$$
, where $l^{simple}$ is your standard critic loss
$$
l^{simple}(s_{t + l}) = [V_\theta(s_{t+l}) - G(s_{t + l})]^2
$$

and $l^{clipped}$ is a clipped version that limits large changes of the value function:
$$
l^{clipped}(s_{t + l}) = [
V_{\theta^{\text{old}}}(s_{t+l}) +
\text{clip}\left(
V_\theta(s_{t+l}) - V_{\theta^\text{old}}(s_{t+l}),
-\text{cliprange}, \text{cliprange}
\right) - G(s_{t + l})] ^ 2
$$

In [ ]:
class PPO:
    def __init__(self, policy, optimizer,
               cliprange=0.2,
               value_loss_coef=0.25,
               max_grad_norm=0.5):
    self.policy = policy
    self.optimizer = optimizer
    self.cliprange = cliprange
    self.value_loss_coef = value_loss_coef
    # Note that we don't need entropy regularization for this env.
    self.max_grad_norm = max_grad_norm

    def policy_loss(self, trajectory, act):
    """ Computes and returns policy loss on a given trajectory. """
    <TODO: implement>

    def value_loss(self, trajectory, act):
    """ Computes and returns value loss on a given trajectory. """
    <TODO: implement>

    def loss(self, trajectory):
    act = self.policy.act(trajectory["observations"], training=True)
    policy_loss = self.policy_loss(trajectory, act)
    value_loss = self.value_loss(trajectory, act)
    return policy_loss + self.value_loss_coef * value_loss

    def step(self, trajectory):
    """ Computes the loss function and performs a single gradient step. """
    <TODO: implement>

Now everything is ready to do training. In one million of interactions it should be possible to 
achieve the total raw reward of about 1500. You should plot this quantity with respect to 
`runner.step_var` &mdash; the number of interactions with the environment. It is highly 
encouraged to also provide plots of the following quantities (these are useful for debugging as well):

* [Coefficient of Determination](https://en.wikipedia.org/wiki/Coefficient_of_determination) between 
value targets and value predictions
* Entropy of the policy $\pi$
* Value loss
* Policy loss
* Value targets
* Value predictions
* Gradient norm
* Advantages

For optimization it is suggested to use Adam optimizer with linearly annealing learning rate 
from 3e-4 to 0 and epsilon 1e-5.

In [ ]:
from math import ceil

NUM_RUNNER_STEPS = 2048
NUM_RUNNER_EPOCHS = 10
NUM_RUNNER_MINIBATCHES = 32
MINIBATCH_SIZE = NUM_RUNNER_STEPS * NENVS // NUM_RUNNER_MINIBATCHES

TOTAL_MINIBATCHES = int(3e6) // NUM_RUNNER_STEPS // NENVS * NUM_RUNNER_EPOCHS * NUM_RUNNER_MINIBATCHES
MINIBATCHES_IN_EPOCH = NUM_RUNNER_EPOCHS * NUM_RUNNER_MINIBATCHES
TOTAL_EPOCHS = ceil(TOTAL_MINIBATCHES // MINIBATCHES_IN_EPOCH)

START = 0
CKPT_FREQ = 3
AGENT_LR = 3e-4

EXPERIMENT_NAME = 'PPO'
config = wandb.config
config.init_lr = AGENT_LR
config.num_runner_steps = NUM_RUNNER_STEPS
config.num_runner_epochs = NUM_RUNNER_EPOCHS
config.num_runner_minibatches = NUM_RUNNER_MINIBATCHES
config.minibatch_size = MINIBATCH_SIZE
config.nenvs = NENVS

MY_NAME = <Your username>

wandb.init(project='ppo', entity=MY_NAME, name=EXPERIMENT_NAME, config=config)

env.reset()

policy = Policy(model)
runner = make_ppo_runner(env, policy, num_runner_steps=NUM_RUNNER_STEPS,
                         num_epochs=NUM_RUNNER_EPOCHS, num_minibatches=NUM_RUNNER_MINIBATCHES)


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=AGENT_LR, eps=1e-5)
sched = torch.optim.lr_scheduler.LambdaLR(optimizer,
                                          lambda epoch: (TOTAL_EPOCHS - epoch) / (TOTAL_EPOCHS - epoch + 1))
ppo = PPO(policy, optimizer)
ppo.step(runner.get_next())

In [ ]:
import os

MODEL_BASE_PATH = os.path.join(CWD, 'models')
os.makedirs(MODEL_BASE_PATH, exist_ok=True)

In [ ]:
from tqdm.notebook import tqdm

for ep in tqdm(range(START, TOTAL_EPOCHS), desc='Epoch'):
    for mb_idx in range(MINIBATCHES_IN_EPOCH):
        env.env.set_step((ep * MINIBATCHES_IN_EPOCH + mb_idx) * MINIBATCH_SIZE)
        trajectory = runner.get_next()
        losses = ppo.step(trajectory)
        for k, v in losses.items():
            env.env.add_summary_scalar(k, v)
        env.env.add_summary_scalar('lr', sched.get_last_lr())
    if (ep + 1) % CKPT_FREQ == 0:
        sched.step()
        state_dict = {'model': model.state_dict(),
                      'optimizer': optimizer.state_dict(),
                      'scheduler': sched.state_dict(),
                      'ep': ep}
        torch.save(state_dict, os.path.join(MODEL_BASE_PATH, 'ppo.pth'))